In [1]:
from create_constraints_neutrinos_fixed import *

In [2]:
import numpy as np

In [54]:
def modify_list(lst):
    lst.append(100)
    return lst

original_list = [1, 2, 3]
print(modify_list(original_list.copy()))  # Prevents mutation of original_list
print(original_list)  # Output: [1, 2, 3]

def modify_dict(dct):
    dct['key2'] = 'value2'
    return dct

original_dict = {'key': 'value'}
print(modify_dict(original_dict.copy()))  # Prevents mutation of original_dict
print(original_dict)  # Output: {'key': 'value'}


[1, 2, 3, 100]
[1, 2, 3]
{'key': 'value', 'key2': 'value2'}
{'key': 'value'}


# Encoder

In [55]:
def encode_assignment(assignment_list=None, assignment_dict=None):
    """
    Given a SAT assignment list, returns a 16x8 matrix representing the assignment, a dictionary mapping each row
    return:
      - matrix: a 16x8 list-of-lists where each cell is 1 (True), 0 (False), or -1 (unassigned)
      - vev_dict: a dictionary mapping each row (12 to 16) to its vev value (0 if off, 1-3 if on, or -1)
      - z2_val, z3_val: the Boolean assignments for auxiliary variables z2 and z3 (as 0/1, or -1 if unassigned).
    """

    if assignment_list is None and assignment_dict is None:
        raise ValueError("Either assignment_list or assignment_dict must be provided.")
    
    if assignment_list is not None and assignment_dict is not None:
        raise ValueError("Only one of assignment_list or assignment_dict should be provided.")
    
    if assignment_list is not None:
        assert isinstance(assignment_list, list), "assignment_list must be a list."
        assignment = {}
        for i in range(len(assignment_list)):
            if assignment_list[i] > 0:
                assignment[assignment_list[i]] = 1
                print(assignment_list[i], variable_to_string(assignment_list[i]), assignment[assignment_list[i]])
            else:
                assignment[-assignment_list[i]] = 0
                print(assignment_list[i], variable_to_string(-assignment_list[i]), assignment[-assignment_list[i]])
    
    if assignment_dict is not None:
        assert isinstance(assignment_dict, dict), "assignment_dict must be a dictionary"
        assignment = assignment_dict.copy()

    print("Assignment: ", assignment)
    for k, v in assignment.items():
        print(k, variable_to_string(k), v)

    # Build the 16x8 matrix.
    matrix = [[-1 for _ in range(COLS)] for _ in range(ROWS)]
    for i in range(1, ROWS+1):
        for j in range(1, COLS+1):
            v = var(i, j)
            if v in assignment:
                matrix[i-1][j-1] = 1 if assignment[v] else 0
            else:
                matrix[i-1][j-1] = -1

    # For rows 12-16, extract the vev value.
    vev_dict = {}
    for r in range(12, 17):
        # The 4th cell (column 4) determines if vev is active.
        cell_val = matrix[r-1][3]  # index 3 corresponds to col4
        if cell_val == -1:
            vev_dict[r] = -1
        elif cell_val == 0:
            vev_dict[r] = 0
        elif cell_val == 1:
            # If cell is 1, then exactly one of the three vev variables should be True.
            chosen = -1
            found = False
            for k in range(1, 4):
                vnum = vev_var(r, k)
                if vnum in assignment:
                    if assignment[vnum]:
                        if found:  # More than one true: ambiguous.
                            chosen = -1
                            found = False
                            break
                        else:
                            chosen = k
                            found = True
                else:
                    # If any vev variable is unassigned, mark the vev value as -1.
                    chosen = -1
                    found = False
                    break
            vev_dict[r] = chosen if found else -1
        else:
            vev_dict[r] = -1

    # Extract z2 and z3.
    z2_val = 1 if (z2 in assignment and assignment[z2]) else (0 if z2 in assignment and not assignment[z2] else -1)
    z3_val = 1 if (z3 in assignment and assignment[z3]) else (0 if z3 in assignment and not assignment[z3] else -1)

    return matrix, vev_dict, z2_val, z3_val

# Decoder

In [97]:
def decode_assignment(matrix, vev_dict, z2_val=-1, z3_val=-1):
    """
    Given the following:
      - matrix: a 16x8 list-of-lists (cells are 0, 1, or -1)
      - vev_dict: a dictionary mapping each row (12 to 16) to its vev value (0, or 1-3, or -1)
      - z2_val and z3_val: the assignments for z2 and z3 (0, 1, or -1)
    Return a list of variable numbers that encode the assignment.
    This covers both the matrix variables and the auxiliary vev variables plus z2 and z3.
    """
    assignment = {}
    # Fill in the matrix variables.
    for i in range(1, ROWS+1):
        for j in range(1, COLS+1):
            assignment[var(i, j)] = matrix[i-1][j-1]
    
    # For rows 12-16, fill in the vev variables.
    for r in range(12, 17):
        v_val = vev_dict.get(r, -1)
        for k in range(1, 4):
            if v_val == -1:
                assignment[vev_var(r, k)] = -1
            elif v_val == 0:
                # If vev is 0 then none of the vev variables are set.
                assignment[vev_var(r, k)] = 0
            else:
                # If vev is k then vev_var(r,k) should be True (1), and the others False.
                assignment[vev_var(r, k)] = 1 if (v_val == k) else 0

    # Fill in z2 and z3.
    assignment[z2] = z2_val
    assignment[z3] = z3_val

    print(assignment)

    # remove all values that are -1
    assignment = {k: v for k, v in assignment.items() if v != -1}

    print(assignment)

    # convert assignment dictionary to assignment list (positive or negative based on value, remove unassigned variables)
    assignment_list = []
    for key in assignment:
        if assignment[key] != -1:
            if assignment[key] == 0:
                assignment_list.append(-key)
            else:
                assignment_list.append(key)

    return assignment_list


## Example

In [57]:
# For demonstration, create a dummy assignment list.
# (In practice, this assignment would be produced by a SAT solver.)
dummy_assignment_list = []

# For Row 1, force columns 1-3 to 0, column 4 to 1, and set column 5=1 (others 0) as an example.
for j in range(1, 4):
    dummy_assignment_list.append(-var(1, j))
dummy_assignment_list.append(var(1, 4))
dummy_assignment_list.append(var(1, 5))

# For rows 12-16, let the 4th cell be 0 (so vev should be 0).
for r in range(12, 17):
    for j in range(1, 4):
        dummy_assignment_list.append(-var(r, j))
    # And set all corresponding vev variables to False.
    for k in range(1, 4):
        dummy_assignment_list.append(-vev_var(r, k))

# sort the dummy assignment list by mod of variable number
dummy_assignment_list.sort(key=lambda x: abs(x))

# ---- Encode: from assignment -> matrix + auxiliary values ----
print("Dummy assignment list: ", dummy_assignment_list)
matrix, vev_dict, z2_val, z3_val = encode_assignment(dummy_assignment_list)
print("Encoded 16x8 matrix:")
print(np.array(matrix))
print("\nEncoded vev values for rows 12-16:")
for r in range(12, 17):
    print(f"Row {r}: vev = {vev_dict.get(r, -1)}")
print("\nEncoded z2 =", z2_val, " z3 =", z3_val)

# ---- Decode: from matrix + auxiliary values -> assignment dictionary ----
decoded_assignment_list = decode_assignment(matrix, vev_dict, z2_val, z3_val)

print("\nDecoded assignment list: ", decoded_assignment_list)
for i in range(len(decoded_assignment_list)):
    if decoded_assignment_list[i] > 0:
        print(variable_to_string(decoded_assignment_list[i]), " = True")
    else:
        print(variable_to_string(-decoded_assignment_list[i]), " = False")

Dummy assignment list:  [-1, -2, -3, 4, 5, -89, -90, -91, -97, -98, -99, -105, -106, -107, -113, -114, -115, -121, -122, -123, -134, -135, -136, -137, -138, -139, -140, -141, -142, -143, -144, -145, -146, -147, -148]
-1 Cell (1, 1) 0
-2 Cell (1, 2) 0
-3 Cell (1, 3) 0
4 Cell (1, 4) 1
5 Cell (1, 5) 1
-89 Cell (12, 1) 0
-90 Cell (12, 2) 0
-91 Cell (12, 3) 0
-97 Cell (13, 1) 0
-98 Cell (13, 2) 0
-99 Cell (13, 3) 0
-105 Cell (14, 1) 0
-106 Cell (14, 2) 0
-107 Cell (14, 3) 0
-113 Cell (15, 1) 0
-114 Cell (15, 2) 0
-115 Cell (15, 3) 0
-121 Cell (16, 1) 0
-122 Cell (16, 2) 0
-123 Cell (16, 3) 0
-134 vev(12, 1) 0
-135 vev(12, 2) 0
-136 vev(12, 3) 0
-137 vev(13, 1) 0
-138 vev(13, 2) 0
-139 vev(13, 3) 0
-140 vev(14, 1) 0
-141 vev(14, 2) 0
-142 vev(14, 3) 0
-143 vev(15, 1) 0
-144 vev(15, 2) 0
-145 vev(15, 3) 0
-146 vev(16, 1) 0
-147 vev(16, 2) 0
-148 vev(16, 3) 0
Assignment:  {1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 89: 0, 90: 0, 91: 0, 97: 0, 98: 0, 99: 0, 105: 0, 106: 0, 107: 0, 113: 0, 114: 0, 115: 0, 12

# Propagator

In [58]:
from pysat.solvers import Solver

# Define the CNF formula
cnf = [[1, 2], [-1, 3], [2, -3]]

# Initialize the solver with the CNF formula
solver = Solver(bootstrap_with=cnf)

# Propagate with a partial assignment: Assign x1 = True (literal 1)
result = solver.propagate(assumptions=[1])

# Display the result of propagation
print("Is propagation successful?", result[0])
print("Propagated literals:", result[1])

# Close the solver
solver.delete()


Is propagation successful? True
Propagated literals: [1, 3, 2]


In [59]:
from pysat.solvers import Solver

# Define the CNF formula
cnf = [[1, -2, 3], [-1, 4], [2, 5], [-3, -4], [4, -5]]

# Initialize the solver with the CNF formula
solver = Solver(bootstrap_with=cnf)

# Propagate with a partial assignment: Assign x1 = True (literal 1)
result = solver.propagate(assumptions=[1])

# Display the result of propagation
print("Is propagation successful?", result[0])
print("Propagated literals:", result[1])

# Close the solver
solver.delete()


Is propagation successful? True
Propagated literals: [1, 4, -3]


In [60]:
from pysat.formula import CNF

def pure_literal_elimination(cnf):
    # Initialize dictionaries to count occurrences
    literal_count = {}
    for clause in cnf.clauses:
        for literal in clause:
            if literal not in literal_count:
                literal_count[literal] = 0
            literal_count[literal] += 1

    # Identify pure literals
    pure_literals = set()
    for literal in literal_count:
        if -literal not in literal_count:
            pure_literals.add(literal)

    # If no pure literals, return the original CNF
    if not pure_literals:
        return cnf, set()

    # Assign truth values to pure literals and simplify the CNF
    new_clauses = []
    assignments = set()
    for clause in cnf.clauses:
        if not any(lit in pure_literals for lit in clause):
            new_clause = [lit for lit in clause if -lit not in pure_literals]
            new_clauses.append(new_clause)
        else:
            # Record the assignment
            for lit in clause:
                if lit in pure_literals:
                    assignments.add(lit)

    # Create a new CNF with simplified clauses
    simplified_cnf = CNF()
    simplified_cnf.clauses = new_clauses

    return simplified_cnf, assignments

def unit_clause_propagation(cnf):
    assignments = set()
    unit_clauses = [clause for clause in cnf.clauses if len(clause) == 1]
    while unit_clauses:
        unit = unit_clauses.pop()
        lit = unit[0]
        assignments.add(lit)
        new_clauses = []
        for clause in cnf.clauses:
            if lit in clause:
                continue  # Clause is satisfied
            if -lit in clause:
                new_clause = [l for l in clause if l != -lit]
                if len(new_clause) == 0:
                    return None, None  # Conflict detected
                new_clauses.append(new_clause)
                if len(new_clause) == 1:
                    unit_clauses.append(new_clause)
            else:
                new_clauses.append(clause)
        cnf.clauses = new_clauses
    return cnf, assignments

def simplify_cnf(cnf):
    total_assignments = set()
    while True:
        cnf, pure_assignments = pure_literal_elimination(cnf)
        if not pure_assignments:
            break
        total_assignments.update(pure_assignments)

    while True:
        cnf, unit_assignments = unit_clause_propagation(cnf)
        if unit_assignments is None:
            return None, None  # Conflict detected
        if not unit_assignments:
            break
        total_assignments.update(unit_assignments)

    return cnf, list(total_assignments)


In [61]:
from pysat.solvers import Glucose3

cnf = [
    [1, 2],
    [-1, 3],
    [-3, -2],
    [4]
]

cnf = CNF(from_clauses=cnf)

simplified_cnf, assignments = simplify_cnf(cnf)

print("Simplified CNF:", simplified_cnf.clauses)
print("Assignments:", assignments)

# Use Glucose3 with the CNF loaded at startup.
with Glucose3(bootstrap_with=cnf) as solver:
    # ---- Example 1: Propagation under a consistent assumption ----
    # Assume x1 is True (i.e. literal 1)
    assumptions = [1]
    # propagate returns a tuple: (satisfiable, [list of propagated literals])
    sat, propagated = solver.propagate(assumptions)
    
    print(f"Propagation with assumption {assumptions}:")
    print("Satisfiable:", sat)
    print("Propagated literals:", propagated)
    # Expected: With x1 True, clause [-1,3] forces x3 to True, and clause [-3,-2]
    # then forces -2 (i.e. x2 False). So, propagated literals should be [3, -2].

    # ---- Example 2: Propagation under conflicting assumptions ----
    # Here, assume x1=True (1) and x3=False (-3).
    # With x1 True, clause [-1,3] forces x3 True, which conflicts with -3.
    # assumptions_conflict = [1, -3]
    # sat_conflict, propagated_conflict = solver.propagate(assumptions_conflict)
    
    # print(f"\nPropagation with conflicting assumptions {assumptions_conflict}:")
    # print("Satisfiable:", sat_conflict)
    # print("Propagated literals:", propagated_conflict)
    # Expected: Conflict is detected (sat_conflict is False) and no propagation is returned.


Simplified CNF: [[1, 2], [-1, 3], [-3, -2]]
Assignments: [4]
Propagation with assumption [1]:
Satisfiable: True
Propagated literals: [1, 3, -2]


In [62]:
# Global

cnf = CNF()
cnf.from_file("constraints16x8_neutrinos_fixed.cnf")
simplified_cnf, assignments_after_simp = simplify_cnf(cnf)
assert simplified_cnf is not None, "Conflict detected in the CNF formula."

In [71]:
###############################
# Propagator: using PySAT's propagate method
###############################

def propagate_assignments(assumptions):
    """
    Given a current (partial) assignment (a dict mapping var numbers -> 1/0),
    use the CNF (with all constraints) and PySAT's unit propagation to
    obtain additional forced assignments.
    
    Returns a new assignment dictionary (the union of current and propagated assignments).
    """
    # read cnf from file
    
    # Open the solver and propagate.
    with Solver(bootstrap_with=simplified_cnf.clauses) as solver:
        # The propagate() method returns a list of literals that are implied.
        # (Some solvers support this method; here we assume it does.)
        propagated = solver.propagate(assumptions=assignments_after_simp+assumptions)

        if not propagated:
            print("Error!")
            return None
    
        # if unsat, find the conflict
        if propagated and propagated[0] == False:
            print("UNSAT")
            print("Conflict:", solver.get_core())
            return None
    
    # Combine the given assumptions list with the propagated literals in a new list. Make a deepcopy of the given assumptions.
    
    return list(set(assignments_after_simp+assumptions+propagated[1]))
    


In [72]:
# For demonstration, let's propagate the dummy assignment list.

dummy_assignment_list = [1, 5, -6, -7, -8, 13, 21, -14, -22, -15, -23, -16, -24, -89, -90, -91, -97, -98, -99, -105, -106, -107, -113, -114, -115, -121, -122, -123]

propagate_assignments(dummy_assignment_list)

UNSAT
Conflict: None


## Example

In [119]:
# Define a partial assignment as an example.
# (In a real application this would be built up during search.)
current_assignment = []
# current_assignment.append(var(1, 5))
# current_assignment.append(var(4, 4))
# current_assignment.append(var(4, 6))
# current_assignment.append(var(7, 1))
# current_assignment.append(var(8, 2))
# current_assignment.append(var(8, 6))

# current_assignment.append(var(10, 7))

# current_assignment.append(var(12, 1))
# current_assignment.append(var(12, 8))
# current_assignment.append(var(13, 4))

# current_assignment.append(-var(14, 1))
# current_assignment.append(-var(14, 2))
# current_assignment.append(-var(14, 3))
# # current_assignment.append(-var(14, 4))

# current_assignment.append(-var(14, 5))
# current_assignment.append(-var(14, 6))
# current_assignment.append(-var(14, 7))
# current_assignment.append(-var(14, 8))

# current_assignment.append(-vev_var(13, 1))
# current_assignment.append(-vev_var(13, 2))

# For row 12, force col4=1 so that vev becomes active.
# current_assignment.append(var(12, 4))
# Also, for row 12, assign one of the vev variables (say, option 2 = True)
# current_assignment.append(vev_var(12, 2))
# And force the other vev variables for row 12 to 0.
# current_assignment.append(-vev_var(12, 1))
# current_assignment.append(-vev_var(12, 3))
# Also assign z2 and z3 arbitrarily.
# current_assignment.append(z2)
# current_assignment.append(z3)

print("Current (partial) assignment:")
print("current_assignment: ", current_assignment)
mat, vevs, z2_val, z3_val = encode_assignment(assignment_list=current_assignment)
print(np.array(mat))
print("vev:", vevs, " z2:", z2_val, " z3:", z3_val)

# Propagate using the CNF.
all_assignments = propagate_assignments(current_assignment)
if all_assignments is None:
    print("Conflict detected.")
else:
    print("\nAssignments removed:", set(current_assignment)-set(all_assignments))
    print("\nNew assignments propagated:", set(all_assignments)-set(current_assignment))
    print("\nAll assignments propagated:", all_assignments)
    print("\nAfter propagation:")
    new_mat, new_vevs, new_z2, new_z3 = encode_assignment(assignment_list=all_assignments)
    print(np.array(new_mat))
    print("vev:", new_vevs, " z2:", new_z2, " z3:", new_z3)

Current (partial) assignment:
current_assignment:  []
Assignment:  {}
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
vev: {12: -1, 13: -1, 14: -1, 15: -1, 16: -1}  z2: -1  z3: -1

Assignments removed: set()

New assignments propagated: {4, 73, 12, 81, -9, 20, -84, -83, -82, -19, -18, -17, -76, -75, -74, -11, -10, -1, -3, -2}

All assignments propagated: [4, 12, 20, -84, -83, -82, -76, -75, -74, 73, 81, -19, -18, -17, -11, -10, -9, -1, -3, -2]

After propagation:
4 Cell (1, 4) 1
12 Cell (2, 4) 1
20 Cell (3, 4) 1
-84 Cell (11, 4) 0
-83 Cell (11, 3) 0
-82 Cell (11, 2) 0
-76 Cell (10, 4) 0
-75 Cell (10, 3) 0
-7

# Legal moves

In [114]:
def get_legal_next_row_values(current_assignment, next_row):
    """
    Given a current (partial) assignment (a list with variable numbers - positive if True, negative if False),
    and a next row index (1-indexed) that we want to fill next,
    return a list of legal candidate assignments for that row.
    
    Each candidate is represented as a list of 8 values (one per column) with 0/1,
    such that when the candidate is added to current_assignment the full CNF is still satisfiable.
    
    This function uses the CNF and, for each candidate among 2^8 possibilities,
    checks if it is extendible.
    """
    legal_candidates = []
    legal_candidates_encoded = []
    
    # Enumerate all 2^8 possible assignments for the 8 cells in the next row.
    for cand in range(2 ** COLS):
        candidate_assumptions = []
        candidate_row = []
        for j in range(1, COLS + 1):
            # Determine the bit for column j. (MSB - leftmost corresponds to column 1.)
            bit = (cand >> (COLS - j)) & 1 
            candidate_row.append(bit)
            candidate_assumptions.append(var(next_row, j) if bit == 1 else -var(next_row, j))
        
        # Combine current assignment assumptions with the candidate for this row.
        all_assumptions = current_assignment + candidate_assumptions
        
        # Use the solver to check if these assumptions are extendible.
        with Solver(bootstrap_with=simplified_cnf.clauses) as solver:
            if solver.solve(assumptions=assignments_after_simp+all_assumptions):
                legal_candidates.append(candidate_row)
                legal_candidates_encoded.append(cand)
    
    return legal_candidates, legal_candidates_encoded


In [121]:
# example usage

# partial matrix

# partial = """[[ 0  0  0  1  1  0  0  0]
#  [ 0  0  0  1  1  0  0  0]
#  [ 0  0  0  1  1  0  0  0]
#  [ 0  0  0  1  0  1  0  0]
#  [ 0  0  0  1  0  1  0  0]
#  [ 0  0  0  1  0  1  0  0]
#  [ 0  0  0  1  0  1  0  0]
#  [ 0  0  0  1  0  1  0  0]]"""

# partial = partial.replace("  ", ",")
# partial = partial.replace("]\n", "],\n")
# print(partial)

# partial = np.array(eval(partial))
# print("Partial matrix:")
# print(partial)

# partial = partial.tolist()

partial = []

print(partial)

# next row index
next_row = len(partial) + 1

# fill rest of the partial matrix with -1
for i in range(next_row, ROWS+1):
    partial.append([-1] * COLS)

print("Partial matrix:")
print(partial)
print(np.array(partial).shape)

# current assignment

current_assignment = decode_assignment(partial, {}, -1, -1)

print("Current assignment:")
print(current_assignment)

# Get legal candidates for the next row.
candidates, candidates_encoded = get_legal_next_row_values(current_assignment, next_row)

print("\nLegal candidates for row", next_row, ":")
for i, candidate in enumerate(candidates):
    print(f"Candidate {i+1}: {candidate} (encoded as {candidates_encoded[i]})")




[]
Partial matrix:
[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1]]
(16, 8)
{1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1, 7: -1, 8: -1, 9: -1, 10: -1, 11: -1, 12: -1, 13: -1, 14: -1, 15: -1, 16: -1, 17: -1, 18: -1, 19: -1, 20: -1, 21: -1, 22: -1, 23: -1, 24: -1, 25: -1, 26: -1, 27: -1, 28: -1, 29: -1, 30: -1, 31: -1, 32: -1, 33: -1, 34: -1, 35: -1, 36: -1, 37: -1, 38: -1, 39: -1, 40: -1, 41: -1, 42: -1, 43: -1, 44: -1, 45: -1, 46: -1, 47: -1, 48: -1, 49: -1, 50: -1, 51: -1, 52: -1, 53: -1, 54: -1, 55: 

In [117]:
# represent 17 as binary with 8 bits list of 0/1
print([int(x) for x in list('{0:0b}'.format(130).zfill(8))])

[1, 0, 0, 0, 0, 0, 1, 0]
